Bienvenidos a la segunda parte de estas notas. El objetivo de este y los siguiente notebooks será de profundizar en el manejo de la computación en paralelo usando CUDA C y aprender (o revisar) algunos tópicos de física estadística.

Al final del notebook se preguntarán si en realidad este no pertenece a la primera parte de las notas, puesto que por el momento no veremos todavía nada de física y nos dedicaremos únicamente a aprender a utilizar algunas herramientas de CUDA. Lo cierto es que estas herramientas son básicas en los ejemplos físicos que daremos, así que no desesperen y aprendamos a generar *números aleatorios* dentro de nuestros `kernels`. 

##  Librería cuRAND

Como ya seguro sabrán, las computadoras no pueden en realidad generar números **aleatorios** *per se*, sino que generan números *pseudoaleatorios* o *cuasialeatorios*. Esto debido al determinismo de la computadora misma. 

Sin embargo los matemáticos y computólogos han trabajado durante décadas para poder crear algoritmos que creen números lo suficientemente *aleatorios*. Hasta hoy, el algoritmo preferido por todos aquellos que trabajan en estadística es el **Mersenne Twister** creado en el Japón a finales del milenio pasado. Para un poco más de información sobre esto revisa las referencias de este notebook.

CUDA ha decidido utilizar en su librería `cuRAND` otros generadores tales como el *XORWOW* o el *SOBOL*. Pero no nos metamos en eso. Hasta hace unas versiones, el generador *MT* no estaba disponible en `device API`. Afortunadamente han corregido eso.

**cuRAND** fue creada para la utilización de `CUDA` en otros ambientes tales como los financieros, matemáticos, físicos donde la estadística puede ser pan de cada día. 

Para todos aquellos ingenuos mal acostumbrados (como nosotros en un principio) que piensan que únicamente basta con poner en el `kernel` algo así como

```C

__global__ void Aleatorio(int n, float *d_A) 
{
    idx = blockDim.x*blockIdx.x + threadIdx.x ;
    
    d_A[idx] = rand() ;
    
}

```

Pues están equívocados. El proceso es más complicado que eso y hay algunos detalles extras que poner.

### Host API y device API

Uno de los primeros detalles a comentar es a la hora de compilar. al usar la librería `cuRAND` uno tiene escribir la bandera `-lcurand` en la línea `nvcc`.

In [ ]:
nvcc mi_primer_programa.cu -lcurand

De la misma manera, en el programa se tiene que incluir la librería escribiendo 
```C
#include <curand.h>
```

Una de las diferencias esenciales entre usar `Host API` y `Device API` es el dinamismo del problema. Si de entrada uno sabe la cantidad de números aleatorios y no desea controlar esta cantidad, `Host API` puede llegar a ser más conveniente y práctico. Sin embargo, si uno no sabe la cantidad de número aleatorios que necesitara y desea tener un mayor control sobre el programa, `Device API`, aunque más tedioso y complejo, traerá mayores ventajas.

+ **Host API**

El procedimiento es en realidad bastante sencillo. 
+ En primer lugar se tiene que crear un generador de números aleaatorios. Este tipo de variable es de tipo `curandRandGenerator_t`.
+ Luego se aloja en el `GPU` la memoria que será destinada a los números aleatorios con `cudaMalloc()`.
+ Aquí viene lo delicado. Hay que definir a nuestro generador usando las funciones `curandCreateGenerator()` y `curandSetPseudoRandomGeneratorSeed`().
+ Se manda la orden de crear una cierta cantidad de números aleatorios en el `GPU` mediante `curandGenerateUniform()`.

En este último paso hay distintas opciones para generar números aleatorios en distintas distribuciones. En nuestra lista utilizamos una distribución uniforme, pero también las hay *normal* bajo `curandGenerateNormal()` o *log-normal* con `curandGenerateLogNormal()`.

Cada una de estas distribuciones tiene la opción de dar un número con precisión `doble` agregando `Double` al final del nombre de la función, como en `curandGenerateLogNormalDouble()`.
También hay la opción de dar dúplas de números aleatorios (bastante útil) agregando un `2` al final del nombre de la función, como en `curandGenerateNormal2()`.

+ Finalmente hay que limpiar todo. Análogamente a `cudaFree()` y `free()`, en `cuRAND` tenemos `curandDestroyGenerator()`.

Mostremos un ejemplo sencillo para irnos acostumbrando a esto.

```C

// Este programa utiliza cuRAND para generar 10 números aleatorios

#include<stdio.h>
#include<stdlib.h>
#include<cuda.h>
#include<curand.h>

int main ( int argc, char ∗argv[] ) {
     int n = 10 ;
     curandGenerator_t gen ; // Creamos la variable gen que será nuestro generador
     float ∗devData, ∗hostData ;

    
     hostData = (float∗)calloc(n, sizeof(float) ) ; // Alocación de 10 floats en el CPU
     cudaMalloc( (void ∗∗) &devData, n∗sizeof(float) ) ; // Alocación de la memoria en el GPU
     
     curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_MTGP32) ; // Creación de un generador MT
     curandSetPseudoRandomGeneratorSeed(gen, 1234ULL) ; // Ajustamos la semilla o seed
     curandGenerateUniform(gen, devData, n) ; // Generamos los números aleatorios en el GPU

     cudaMemcpy( hostData, devData, n∗sizeof(float), cudaMemcpyDeviceToHost ) ;

      // Unas líneas para mostrar los resultados
     printf( ”Distribucion uniforme entre 0. y 1.:\n” ) ;
     for( int i = 0 ; i < n ; i ++) 
     {
         printf(”%1.4f\n”, hostData[i]) ;
     }
     printf( ”\n” ) ;

     curandDestroyGenerator(gen) ;
     cudaFree(devData) ;
     free(hostData) ;
}


```

Este código lo colocamos en un archivo llamado host_api.cu

In [ ]:
nvcc host_api.cu -lcurand -o host_api

In [ ]:
./host_api

+ **Device API**

Como mencionamos anteriormente, `Device API` nos da el control sobre todos los generadores y "números aleatorios" que creamos en el `GPU`. Pero mayores poderes conllevan mayores responsabilidades, por lo que el código se vuelve a primera vista un poco más complicado, sin embargo la idea de fondo es bastante sencilla.

A diferencia de `Host API`, en este caso habrá que escribir un `kernel` el cual se encarge de iniciar un *generador* distinto en cada `thread` que vayamos a ocupar. Este `kernel` tiene la forma siguiente:

```C

__global__ Init( curandState * state) 
{
    idx = blockDim.x*blockIdx.x + threadIdx.x ;
    curandinit( 1234, idx, 0, state[idx]) ;
}
```

Veamos. Los generadores que había en nuestro ejemplo `Host API` han sido remplazados por las variables tipo `State`. En este ejemplo hemos usado un *generador de números pseudo-aleatorios* (el predefinido para `Device API` es *XORWOW*, para *MT* lo haremos más adelante) . La variable se declara en el `host` como `curandState`. Claro hay también otro tipo de generadores, que pueden ser consultados en la documentación.

La segunda cosa que salta a la vista es la función `curandinit()` la cual coloca en cada `thread` un generador. Los cuatro argumentos de `curandinit()` se refieren a:
+ la `semilla` o `seed` del generador.
+ la sub-secuencia. Esto es lo que nos permite asegurar que cada `thread` no generará la misma cadena de números aleatorios.
+ el `offset`, el cual para fines prácticos lo anularemos.
+ el generador que se le asigna a cada una de los `threads`.



+ segundo kernel
+ como se ve el int main
+ MT

## Referencias

+ La documentación completa de cuRAND se encuentra [aquí](http://hpc.oit.uci.edu/nvidia-doc/sdk-cuda-doc/CUDALibraries/doc/CURAND_Library.pdf)
+ Sobre [Pseudo-aleatoriedad](https://en.wikipedia.org/wiki/Pseudorandomness) y [Generadores de Números Pseudo-aleatorios (PNG)](https://en.wikipedia.org/wiki/Pseudorandom_number_generator)
+ Sobre [Cuasi-aleatoriedad](https://en.wikipedia.org/wiki/Low-discrepancy_sequence). También [aquí](http://mathworld.wolfram.com/QuasirandomSequence.html) hay más info.
+ Un poco más de información sobre Mersenne Twister en su [Wiki](https://en.wikipedia.org/wiki/Mersenne_Twister)